In [1]:
#Inspired from -
# https://github.com/MVIG-SJTU/AlphaPose
# https://github.com/Hzzone/pytorch-openpose

In [2]:
import sys
sys.path.append('posedet_alpha')
import time
from PIL import Image
from detector_alpha import PoseDet
from trackers.PoseFlow.poseflow_infer import PoseFlowWrapper
import cv2
import json
import numpy as np
import glob
from tqdm import tqdm
from matplotlib import colors
from collections import defaultdict
from trackers.tracker_api import Tracker

def hex2rgb(_hex):
    _hex = _hex.lstrip('#')
    return [int(_hex[i:i+2],16) for i in (0,2,4)]

def process_poses(out):
    pose = []
    pscores = []
    _boxes= []
    for key in range(len(out)-1) if pose!=None else []:
        pose.append(out[key]['keypoints'])
        pscores.append(float(out[key]['proposal_score']))
        _boxes.append(out[key]['bbox'])
    _boxes = np.array(_boxes,dtype=np.float32)
    pose   = np.array(pose,dtype=np.float32)
    return pose, pscores, _boxes

colors = [hex2rgb(val) for val in ['#FF0000','#00FF00','#0000FF','#FFFF00','#00FFFF','#FF00FF','#000000','#FFFFFF']+list(colors.TABLEAU_COLORS.values())]
files  = glob.glob('/home/irfan/Desktop/Data/Pose_JSON_Data/ShanghaiTech/gt/test/frames/01_0015/*')
#cap  = cv2.VideoCapture(src)

demo = PoseDet()
pf   = PoseFlowWrapper(link=25,match=0.2)
tcr  = Tracker()
i    = 0
out_dict = defaultdict(dict)
results = []
for file in tqdm(sorted(files)):
    i+=1
    init_time = time.time()
    img       = cv2.imread(file)
    #if i<200: continue
    
    img   = cv2.resize(img,(256,192))
    _im= img.copy()
    pose,inps,boxes,cropped_boxes  = demo.detect(img,show=True)
    
    if pose != None and len(pose):
        out = pf.step(img,{'result':pose,'imgname':i})
        for key in range(len(out)-1) if pose!=None else []:
            ps = out[key]
            for pt in ps['keypoints']:
                pid = ps['new_pid']
                cv2.putText(img,f"id : {pid}",[int(ps['bbox'][0]),int(ps['bbox'][1]-10)],cv2.FONT_HERSHEY_COMPLEX,0.5,colors[pid],1)
                cv2.circle(img,pt.astype('int32'),3,colors[pid],1)
    
            out_dict[ps['new_pid']].update( 
                {str(i-1).zfill(4)  : 
                 {'keypoints' : np.concatenate([ps['keypoints'],ps['kp_score']],axis=1).reshape(-1).tolist(),
                  'score' : float(ps['proposal_score'])}
                })
    #pose, pscores, _boxes = process_poses(out)
    #tc = tcr.update(img,inps=inps,bboxs=boxes,pose=pose,cropped_boxes=cropped_boxes,pscores=np.array(pscores,dtype=np.float32))
    #print(tc)
    results.append(Image.fromarray(img))
    cv2.imshow('vis', img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cv2.destroyAllWindows()

Load SE Resnet...


/home/irfan/Desktop/Code/Linux-IO/python_38/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/irfan/Desktop/Code/Linux-IO/python_38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading pose model from ../models/pose/alpha_pose/fast_421_res152_256x192.pth...


/home/irfan/Desktop/Code/Deep-Learning-For-Computer-Vision/pose-detection/posedet_alpha/trackers/utils/utils.py:733: UserWarning: The pretrained weights "../models/pose/alpha_pose/osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth" cannot be loaded, please check the key names manually (** ignored and continue **)
  warnings.warn(


Start pose tracking...



  0%|                                                   | 0/433 [00:00<?, ?it/s]

Loading YOLOX-X model..


100%|█████████████████████████████████████████| 433/433 [07:43<00:00,  1.07s/it]


In [7]:
results[0].save("results/result_3_50.gif", format="GIF", append_images=results[::3],save_all=True, duration=50, loop=1)

In [5]:
len(results[::3])

145

### Experimental codes dumped below

In [ ]:
import sys
sys.path.append('/deep-person-reid/')
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_x1_0',
    model_path='../models/pose/reid/model.pth.tar',
    device='cuda'
)

image_list = [
    
    'a/b/c/image001.jpg',
    'a/b/c/image002.jpg',
    'a/b/c/image003.jpg',
    'a/b/c/image004.jpg',
    'a/b/c/image005.jpg'
]

features = extractor(image_list)
print(features.shape) # output (5, 512)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import copy
import numpy as np
import torch

from posedet import model
from posedet import util
from posedet.body import Body
from posedet.hand import Hand


body_estimation = Body('../models/pose/body_pose_model.pth',device='cuda')

print(f"Torch device: {torch.cuda.get_device_name()}")
src = '/home/irfan/Desktop/Data/shanghaitech/testing/frames/01_0028.avi'
cap = cv2.VideoCapture(src)
cap.set(3, 320)
cap.set(4, 240)

class PoseDet():
    def __init__(self):
        self.body_estimation = Body('../models/pose/body_pose_model.pth',device='cuda')
        #self.hand_estimation = Hand('../models/pose/hand_pose_model.pth')
        
while True:
    ret, oriImg = cap.read()
    oriImg = cv2.resize(oriImg,(480,320))
    candidate, subset = body_estimation(oriImg)
    canvas = copy.deepcopy(oriImg)
    canvas = util.draw_bodypose(canvas, candidate, subset)
    '''
    # detect hand
    hands_list = util.handDetect(candidate, subset, oriImg)

    all_hand_peaks = []
    for x, y, w, is_left in hands_list:
        peaks = hand_estimation(oriImg[y:y+w, x:x+w, :])
        peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
        peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)
        all_hand_peaks.append(peaks)

    canvas = util.draw_handpose(canvas, all_hand_peaks)
    '''
    #cv2.imshow('can', candidate[:,:,-1])#一个窗口用以显示原视频
    #cv2.imshow('sub', subset[:,:,-1])#一个窗口用以显示原视频
    cv2.imshow('demo', canvas)#一个窗口用以显示原视频
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
plt.imshow(_img.permute(1,2,0)+0.5)
plt.show()

In [ ]:
plt.imshow(_im)#.permute(1,2,0))
plt.show()

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(_im)
plt.show()

In [ ]:
_img.shape

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
_img.min(),_img.max()

In [ ]:
tcr = Tracker()

In [ ]:
test_transform

In [ ]:
with open('out.json','w') as file:
    json.dump(out_dict,file)

In [ ]:
[ps['bbox'].astype('uint16') for ps in pose]

In [ ]:
import sys
sys.path.append('posedet_alpha')
from trackers.tracker_api import Tracker


In [ ]:
from ReidModels.osnet_ain import osnet_ain_x1_0

In [ ]:
m = osnet_ain_x1_0(num_classes=1,pretrained=False)

In [ ]:
load_pretrained_weights

In [ ]:
m

In [ ]:
for idx in range(5,-1,-1):
    print(idx)

In [ ]:
for key in pf.track.keys():
    print(pf.track[key].num_boxes)